# Adding Regularization to Prevent Overfitting

### Regularization is a technique used to avoid overfitting. Overfitting happens when your model does well on the data it is trained on, but worse on test data that the model has not seen before, such as live traffic. Overfitting generally occurs when a model is excessively complex, such as having too many parameters relative to the number of observed training data. Regularization allows for you to control your model's complexity and makes the model more generalizable to unseen data.

https://stats.stackexchange.com/questions/228763/regularization-methods-for-logistic-regression

In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys

from six.moves import urllib
import tensorflow as tf

DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult'
TRAINING_FILE = 'adult.data'
TRAINING_URL = '%s/%s' % (DATA_URL, TRAINING_FILE)
EVAL_FILE = 'adult.test'
EVAL_URL = '%s/%s' % (DATA_URL, EVAL_FILE)
data_dir = 'census_data'
model_dir = 'model_dir'

_CSV_COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]

_CSV_COLUMN_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
[0], [0], [0], [''], ['']]

_NUM_EXAMPLES = {
    'train': 32561,
    'validation': 16281,
}

In [32]:
if not tf.gfile.Exists(data_dir):
    tf.gfile.MkDir(data_dir)

In [33]:
def _download_and_clean_file(filename, url):
    """Downloads data from url, and makes changes to match the CSV format."""
    temp_file, _ = urllib.request.urlretrieve(url)
    with tf.gfile.Open(temp_file, 'r') as temp_eval_file:
        with tf.gfile.Open(filename, 'w') as eval_file:
            for line in temp_eval_file:
                line = line.strip()
                line = line.replace(', ', ',')
                if not line or ',' not in line:
                    continue
                if line[-1] == '.':
                    line = line[:-1]
                line += '\n'
                eval_file.write(line)
    tf.gfile.Remove(temp_file)

In [34]:
training_file_path = os.path.join(data_dir, TRAINING_FILE)
_download_and_clean_file(training_file_path, TRAINING_URL)

In [35]:
eval_file_path = os.path.join(data_dir, EVAL_FILE)
_download_and_clean_file(eval_file_path, EVAL_URL)

In [41]:
education = tf.feature_column.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

relationship = tf.feature_column.categorical_column_with_vocabulary_list(
    'relationship', [
        'Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried',
        'Other-relative'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

# To show an example of hashing:
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    'occupation', hash_bucket_size=1000)


Base Continuous Feature Columns

In [42]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')


In [43]:
age_buckets = tf.feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

In [44]:
education_x_occupation = tf.feature_column.crossed_column(
    ['education', 'occupation'], hash_bucket_size=1000)
age_buckets_x_education_x_occupation = tf.feature_column.crossed_column(
    [age_buckets, 'education', 'occupation'], hash_bucket_size=1000)


In [45]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
    """Generate an input function for the Estimator."""
    assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop('income_bracket')
        return features, tf.equal(labels, '>50K')

    # Extract lines from input files using the Dataset API.
    dataset = tf.data.TextLineDataset(data_file)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

    dataset = dataset.map(parse_csv, num_parallel_calls=5)

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [46]:
base_columns = [
    education, marital_status, relationship, workclass, occupation,
    age_buckets,
]
crossed_columns = [
    tf.feature_column.crossed_column(
        ['education', 'occupation'], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        [age_buckets, 'education', 'occupation'], hash_bucket_size=1000),
]

if not tf.gfile.Exists(model_dir):
    tf.gfile.MkDir(model_dir)
model = tf.estimator.LinearClassifier(
    model_dir=model_dir, feature_columns=base_columns + crossed_columns)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_is_chief': True, '_session_config': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f22c81438>, '_model_dir': 'model_dir'}


INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_is_chief': True, '_session_config': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f22c81438>, '_model_dir': 'model_dir'}


In [50]:
model.train(input_fn=lambda: input_fn('census_data/adult.data', 2, True, 20))

Parsing census_data/adult.data
INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-3258


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-3258


INFO:tensorflow:Saving checkpoints for 3259 into model_dir/model.ckpt.


...

INFO:tensorflow:step = 6459, loss = 8.49533 (0.226 sec)


INFO:tensorflow:Saving checkpoints for 6515 into model_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 6515 into model_dir/model.ckpt.


INFO:tensorflow:Loss for final step: 0.809074.


INFO:tensorflow:Loss for final step: 0.809074.


In [51]:
results = model.evaluate(input_fn=lambda: input_fn(
    'census_data/adult.test', 1, False, 20))
for key in sorted(results):
    print('%s: %s' % (key, results[key]))

Parsing census_data/adult.test


INFO:tensorflow:Starting evaluation at 2018-01-06-12:05:32


INFO:tensorflow:Starting evaluation at 2018-01-06-12:05:32


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-6515


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-6515


INFO:tensorflow:Finished evaluation at 2018-01-06-12:05:35


INFO:tensorflow:Finished evaluation at 2018-01-06-12:05:35


INFO:tensorflow:Saving dict for global step 6515: accuracy = 0.835821, accuracy_baseline = 0.763774, auc = 0.883184, auc_precision_recall = 0.694255, average_loss = 0.352014, global_step = 6515, label/mean = 0.236226, loss = 7.03207, prediction/mean = 0.231491


INFO:tensorflow:Saving dict for global step 6515: accuracy = 0.835821, accuracy_baseline = 0.763774, auc = 0.883184, auc_precision_recall = 0.694255, average_loss = 0.352014, global_step = 6515, label/mean = 0.236226, loss = 7.03207, prediction/mean = 0.231491


accuracy: 0.835821
accuracy_baseline: 0.763774
auc: 0.883184
auc_precision_recall: 0.694255
average_loss: 0.352014
global_step: 6515
label/mean: 0.236226
loss: 7.03207
prediction/mean: 0.231491


In [55]:
model = tf.estimator.LinearClassifier(
    model_dir=model_dir, feature_columns=base_columns + crossed_columns,
    optimizer=tf.train.FtrlOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=1.0,
        l2_regularization_strength=1.0))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_is_chief': True, '_session_config': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f19559f60>, '_model_dir': 'model_dir'}


INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_is_chief': True, '_session_config': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f19559f60>, '_model_dir': 'model_dir'}


In [56]:
model.train(input_fn=lambda: input_fn('census_data/adult.data', 4, True, 20))

Parsing census_data/adult.data
INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into model_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into model_dir/model.ckpt.


INFO:tensorflow:step = 1, loss = 13.8629


...

INFO:tensorflow:step = 6501, loss = 3.73303 (0.225 sec)


INFO:tensorflow:Saving checkpoints for 6513 into model_dir/model.ckpt.


INFO:tensorflow:Saving checkpoints for 6513 into model_dir/model.ckpt.


INFO:tensorflow:Loss for final step: 3.11208.


INFO:tensorflow:Loss for final step: 3.11208.


In [ ]:
results = model.evaluate(input_fn=lambda: input_fn(
    'census_data/adult.test', 1, False, 20))
for key in sorted(results):
    print('%s: %s' % (key, results[key]))

Parsing census_data/adult.test


INFO:tensorflow:Starting evaluation at 2018-01-06-12:09:38


INFO:tensorflow:Starting evaluation at 2018-01-06-12:09:38


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-6513


INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-6513


INFO:tensorflow:Finished evaluation at 2018-01-06-12:09:41


INFO:tensorflow:Finished evaluation at 2018-01-06-12:09:41


INFO:tensorflow:Saving dict for global step 6513: accuracy = 0.837295, accuracy_baseline = 0.763774, auc = 0.885742, auc_precision_recall = 0.699091, average_loss = 0.34877, global_step = 6513, label/mean = 0.236226, loss = 6.96726, prediction/mean = 0.243663


INFO:tensorflow:Saving dict for global step 6513: accuracy = 0.837295, accuracy_baseline = 0.763774, auc = 0.885742, auc_precision_recall = 0.699091, average_loss = 0.34877, global_step = 6513, label/mean = 0.236226, loss = 6.96726, prediction/mean = 0.243663


accuracy: 0.837295
accuracy_baseline: 0.763774
auc: 0.885742
auc_precision_recall: 0.699091
average_loss: 0.34877
global_step: 6513
label/mean: 0.236226
loss: 6.96726
prediction/mean: 0.243663
